In [1]:
import os
import sys
import delta
from pyspark.sql.window import Window
import pyspark.sql.functions as psf
import pandas as pd
import pyspark.sql
from pyspark.sql.types import StructType, StructField, DateType, \
    StringType, DoubleType, TimestampType

sys.path.insert(0, os.path.abspath('./source/Producto_DIA_DP'))

from SparkDBUtils import SparkDB
sparkdb = SparkDB()
spark = sparkdb.spark

In [4]:
spark.sql("show schemas").show(truncate=False)

+----------------+
|namespace       |
+----------------+
|analisis_precios|
|default         |
|producto_dia    |
+----------------+



In [5]:
spark.sql("show tables in producto_dia").show(truncate=False)

+------------+---------------+-----------+
|namespace   |tableName      |isTemporary|
+------------+---------------+-----------+
|producto_dia|producto_dim   |false      |
|producto_dia|sequences_cfg  |false      |
|producto_dia|staging_product|false      |
+------------+---------------+-----------+



In [2]:
df = spark.table("producto_dia.staging_product").toPandas()

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id_producto  0 non-null      object
 1   url_product  0 non-null      object
 2   index        0 non-null      int32 
dtypes: int32(1), object(2)
memory usage: 0.0+ bytes


In [4]:
df = spark.table("producto_dia.producto_dim").toPandas()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 773 entries, 0 to 772
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   product_id  0 non-null      object        
 1   product     0 non-null      object        
 2   brand       0 non-null      object        
 3   categories  0 non-null      object        
 4   units       0 non-null      object        
 5   price       0 non-null      float64       
 6   unit_price  0 non-null      float64       
 7   discount    0 non-null      float64       
 8   date        0 non-null      object        
 9   ts_load     0 non-null      datetime64[ns]
dtypes: datetime64[ns](1), float64(3), object(6)
memory usage: 60.5+ KB


In [5]:
df.head()

,product_id,product,brand,categories,units,price,unit_price,discount,date,ts_load
0,None,None,None,None,None,NaN,NaN,NaN,None,NaT
1,None,None,None,None,None,NaN,NaN,NaN,None,NaT
2,None,None,None,None,None,NaN,NaN,NaN,None,NaT
3,None,None,None,None,None,NaN,NaN,NaN,None,NaT
4,None,None,None,None,None,NaN,NaN,NaN,None,NaT


# DELETE producto_dia.staging_product

In [2]:
dt = delta.DeltaTable.forName(spark, "producto_dia.staging_product")

In [3]:
dt.delete(psf.column("index") > "112")

In [4]:
dt.toDF().show()

+-----------+--------------------+-----+
|id_producto|         url_product|index|
+-----------+--------------------+-----+
|      10065|https://www.dia.e...|    0|
|     101846|https://www.dia.e...|    1|
|     102900|https://www.dia.e...|    2|
|     104003|https://www.dia.e...|    3|
|     105302|https://www.dia.e...|    4|
|     106144|https://www.dia.e...|    5|
|     107989|https://www.dia.e...|    6|
|        110|https://www.dia.e...|    7|
|     111575|https://www.dia.e...|    8|
|     112829|https://www.dia.e...|    9|
|     114234|https://www.dia.e...|   10|
|     115160|https://www.dia.e...|   11|
|     116100|https://www.dia.e...|   12|
|     117243|https://www.dia.e...|   13|
|     118963|https://www.dia.e...|   14|
|      11964|https://www.dia.e...|   15|
|     120311|https://www.dia.e...|   16|
|     122086|https://www.dia.e...|   17|
|     123844|https://www.dia.e...|   18|
|     125055|https://www.dia.e...|   19|
+-----------+--------------------+-----+
only showing top

# DELETE producto_dia.producto_dim

In [11]:
dt = delta.DeltaTable.forName(spark, "producto_dia.producto_dim")

In [9]:
dt.delete()

In [13]:
dt.toDF().show()

+----------+-------+-----+----------+-----+-----+----------+--------+----+-------+
|product_id|product|brand|categories|units|price|unit_price|discount|date|ts_load|
+----------+-------+-----+----------+-----+-----+----------+--------+----+-------+
+----------+-------+-----+----------+-----+-----+----------+--------+----+-------+

